
Initial CSV file read in and data clean-up. All NAN rows dropped, less than 1500 rows of 19500. 
Quick spot check found not more than 3 data points missing within a month when data dropped. 

In [1]:
import pandas as pd
import os

In [2]:
hawaii_measurements=pd.read_csv('Resources/hawaii_measurements.csv')
hawaii_measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [3]:
# check nan rows
nans = lambda hawaii_measurements: hawaii_measurements[hawaii_measurements.isnull().any(axis=1)]
nan_measurement_df = pd.DataFrame(nans(hawaii_measurements))
nan_measurement_df.reset_index(drop=True).head()

,station,date,prcp,tobs
0,USC00519397,2010-01-06,NaN,73
1,USC00519397,2010-01-30,NaN,70
2,USC00519397,2010-02-03,NaN,67
3,USC00519397,2010-02-19,NaN,63
4,USC00519397,2010-03-11,NaN,73


In [4]:
#new dataframe with dropped nan percipitation values(7%of data spread throughout dates. )
new_hawaii_measurements=pd.DataFrame(hawaii_measurements.dropna(how='any'))
new_hawaii_measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [5]:
new_hawaii_measurements.to_csv('Resources/new_hawaii_measurements.csv', index=False, header = True)

In [6]:
csv_station=pd.read_csv('Resources/hawaii_stations.csv')
csv_station

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


### SQLAlchemy and SQLite DB

In [7]:
# Dependencies
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [8]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [9]:
# Create a connection to the engine called `conn`
conn = engine.connect()


In [10]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'hawaii_measurement'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    

class Stations(Base):
    __tablename__ = 'hawaii_station'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [11]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [12]:
# Load the cleaned csv file into a pandas dataframe
new_hawaii_measurements = pd.read_csv('Resources/new_hawaii_measurements.csv')
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
measurement_data = new_hawaii_measurements.to_dict(orient='records')
print(measurement_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [13]:
csv_station=pd.read_csv('Resources/hawaii_stations.csv')
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
station_data = csv_station.to_dict(orient='records')
print(station_data[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [14]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [15]:
# Save the reference to the `customers` table as a variable called `table`
measurements_table = sqlalchemy.Table('hawaii_measurement', metadata, autoload=True)
stations_table = sqlalchemy.Table('hawaii_station', metadata, autoload=True)

In [16]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [17]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurements_table.insert(), measurement_data)
conn.execute(stations_table.insert(), station_data)

In [18]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from hawaii_measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [19]:
conn.execute("select * from hawaii_station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]